# Calculating Metrics for Assessing Mortality Data Quality

The following code demonstrates some simple examples of calculating metrics that are part of the Data Quality Assessment Framework and  Jurisdictional Playbook.

## Calculating Metrics Related to Unsuitable Underlying Cause of Death

The following code demonstrates how mortality records can be evaluated against a list of unsuitable underlying cause of death codes. The code

1. Imports some Python libraries
2. Loads some synthetic sample data from a CSV file
3. Loads a list of unsuitable underlying cause of death codes from another CSV file
4. Evaluates the underlying cause of death codes in the sample data against the list of unsuitable underlying cause of death codes
5. Looks at the results on a per-certifier basis

In [ ]:
import os
import pandas as pd
from IPython.display import display

# Load the synthetic death records data, making sure we handle N/A values in a consistent way
death_records = pd.read_csv("./data/NotionalDeathRecordData.csv", keep_default_na=False, na_values=[""])
display(death_records)

In [ ]:
# Load the unsuitable causes of death data; list of unsuitable codes and related category and display
# string are taken directly from Flagg 2021: https://stacks.cdc.gov/view/cdc/100414
unsuitable_causes = pd.read_csv("./data/unsuitable_COD_codes.csv")
display(unsuitable_causes)

In [ ]:
# Extract the unsuitable codes
unsuitable_codes = unsuitable_causes["code"].values

# Function to check if any unsuitable code is a prefix to the code in the record
def is_unsuitable(code):
    return any(code.startswith(unsuitable) for unsuitable in unsuitable_codes)

# Create a new column that is True when the underlying COD is unsuitable
death_records["Unsuitable Underlying"] = death_records["Underlying COD"].apply(is_unsuitable)
display(death_records)

In [ ]:
# Calculate the proportion of records with an unsuitable underlying cause of death and print
proportion = death_records["Unsuitable Underlying"].mean()
print(f"The proportion of records with an unsuitable underlying cause of death is {proportion:.2f}")

In [ ]:
# Group the records by certifier and calculate the proportion of unsuitable records for each certifier and print
certifier_proportions = death_records.groupby("Certifier Name", as_index=False)["Unsuitable Underlying"].mean()
display(certifier_proportions)

In [ ]:
# Filter for certifiers that have unsuitable underlying causes in records they've reported
filtered = certifier_proportions[certifier_proportions['Unsuitable Underlying'] > 0.2]
display(filtered)

In [ ]:
# Print the certifier names in the resulting set; this could drive an automated notification process
for certifier_name in filtered['Certifier Name']:
    print(certifier_name)

In [ ]:
# TODO: Consider listing all of the matching records for each certifier for use in reachback;
# when doing that it might be helpful to include jurisdiction literals in our synthetic data

## Calculating Metrics Related to Certification Within Required Time

The following code demonstrates how mortality records can be evaluated against certification time guidelines. This code uses the same python libraries and sample code set up above and

1. Evaluates the records against a 5 day certification requirement
2. Looks at the results on a per-certifier basis

In [ ]:
# First parse the relevant dates in the sample data from strings into a native datetime
death_records["Date of Death"] = pd.to_datetime(death_records["Date of Death"])
death_records["Date Certified"] = pd.to_datetime(death_records["Date Certified"])

# Calculate the difference in days between the Date Certified and the Date of Death                                                                                  
death_records["Days Difference"] = (death_records["Date Certified"] - death_records["Date of Death"]).dt.days

# Create a column that is True when the time between death and certification is not within 5 days
death_records["Not Within 5 Days"] = ~death_records["Days Difference"].between(0, 5)
display(death_records)

In [ ]:
# Calculate the proportion of records where the Date Certified is not within 5 days of the Date of Death
proportion = death_records["Not Within 5 Days"].mean()
print(f"The proportion of records where the Date Certified is not within 5 days of the Date of Death is {proportion:.2f}")

In [ ]:
# Group the records by certifier and calculate the proportion of late records for each certifier and print
certifier_proportions = death_records.groupby("Certifier Name", as_index=False)["Not Within 5 Days"].mean()

# Filter for certifiers that have a significant proportion of late records for records they've reported
filtered = certifier_proportions[certifier_proportions['Not Within 5 Days'] > 0.06]
display(filtered)

## Calculating Metrics Related to Incomplete Demographic Information

The following code demonstrates how mortality records can be evaluated against metrics for completeness of demographic information. This code uses the same python libraries and sample code set up above and

1. Evaluates the records for missing demographic information
2. Looks at the results on a per-certifier basis

In [ ]:
# Define the demographic columns we're interested in
demographic_columns = [
    "Sex",
    "Age",
    "Bridged Race",
    "Hispanic Origin",
]  # add or remove columns as needed

# Create a new column that is True when at least one demographic field is empty
death_records["Incomplete Demographics"] = (death_records[demographic_columns].isna().any(axis=1))

# Calculate the proportion of records with incomplete demographic fields
proportion = death_records["Incomplete Demographics"].mean()
print(f"The proportion of records with at least one demographic field incomplete is {proportion:.2f}")

In [ ]:
# The Hispanic Origin column may also have an explicit "Unknown" value of 9, which we can check for separately
# Create a new column that is True when at least one demographic field is empty
death_records["Unknown Hispanic Origin"] = (death_records["Hispanic Origin"] == 9)
proportion = death_records["Unknown Hispanic Origin"].mean()
print(f"The proportion of records with an unknown Hispanic Origin value is {proportion:.2f}")

In [ ]:
# Group the records by certifier and calculate the proportion of "incomplete" records for each certifier
# looking at both empty fields and unknown value for Hispanic Origin
certifier_proportions = death_records.groupby("Certifier Name", as_index=False).apply(
    lambda group: (group["Incomplete Demographics"] & group["Unknown Hispanic Origin"]).mean(),
    include_groups=False
)
certifier_proportions.columns = ['Certifier Name', 'Incomplete or Unknown']

# Filter for certifiers that have a significant proportion of "incomplete" records
filtered = certifier_proportions[certifier_proportions['Incomplete or Unknown'] > 0.05]
display(filtered)